In [21]:
import numpy as np
import pandas as pd
import scipy.stats as stats
from scipy.signal import welch
import os

def process_emg_folder(input_folder, output_folder, shuffle_order_file):
    """
    遍历 input_folder 下的所有 CSV 文件，处理 EMG 和 IMU 数据，并保存特征矩阵和标签到 output_folder。
    """
    # 设定采样率
    fs = 1000  # 采样率 Hz
    cycle_duration = 10  # 每个周期 10 秒
    skip_seconds = 5  # 跳过前 4 秒
    use_seconds = 5  # 需要保留的秒数

    cycle_samples = fs * cycle_duration  # 10 秒数据点数 = 10000
    skip_samples = fs * skip_seconds  # 跳过 4 秒 = 4000
    use_samples = fs * use_seconds  # 取后 6 秒 = 6000

    # 滑动窗口参数
    window_size = 200  # 200ms = 200 采样点
    step_size = 100  # 100ms = 100 采样点
    num_windows = (use_samples - window_size) // step_size + 1  # 计算窗口数

    # 创建存储文件夹
    os.makedirs(output_folder, exist_ok=True)

    # **读取 shuffle_order.xlsx**
    shuffle_df = pd.read_excel(shuffle_order_file, engine="openpyxl")
    print(shuffle_df.shape)
    
    if shuffle_df.shape[0] < 15:
        raise ValueError("标签文件数据不足 15 组！请检查 `shuffle_order.xlsx`。")

    # 存储所有数据和标签
    all_features = []
    all_labels = []

    # **遍历 CSV 文件**
    csv_files = sorted([f for f in os.listdir(input_folder) if f.endswith(".csv")])

    for file_idx, file_name in enumerate(csv_files):
        file_path = os.path.join(input_folder, file_name)
        print(f"Processing {file_name} ({file_idx+1}/{len(csv_files)})")

        # **读取数据**
        raw_data = pd.read_csv(file_path)

        # **检查通道数（忽略时间列）**
        num_channels = raw_data.shape[1] - 1
        print(f"Detected {num_channels} channels (excluding time column).")

        # **归一化（Z-score 标准化）**
        data = raw_data.iloc[:, 1:]  # 去掉时间列
        mean_vals = data.mean(axis=0)
        std_vals = data.std(axis=0)
        normalized_data = (data - mean_vals) / (std_vals + 1e-10)  # 避免除零错误

        # **更新原始数据**
        raw_data.iloc[:, 1:] = normalized_data.astype(np.float64)

        segments = []  # 存储所有分割后的数据
        labels = shuffle_df.iloc[file_idx].values.tolist()  # 获取该文件对应的 26 组标签
        num_cycles = 26  # 每个文件固定 26 段（每段 10s）

        for i in range(num_cycles):
            start_idx = i * cycle_samples + skip_samples  # 跳过前 4 秒
            end_idx = start_idx + use_samples  # 取后 6 秒

            if end_idx > len(raw_data):  # 处理不足 6000 采样点的情况
                segment = raw_data.iloc[start_idx:].values  # 取剩余数据
                pad_size = use_samples - len(segment)  # 计算填充数
                segment = np.pad(segment, ((0, pad_size), (0, 0)), mode='constant', constant_values=0)  # 填充 0
            else:
                segment = raw_data.iloc[start_idx:end_idx].values  # 正常提取数据

            # **滑动窗口**
            windows = [
                segment[j:j + window_size, 1:]  # 取 200 采样点，忽略时间列
                for j in range(0, use_samples - window_size + 1, step_size)  # 滑动步长 100
            ]
            segments.append(np.array(windows))

        # **转换为 NumPy 数组**
        segments_array = np.array(segments)  # 形状 (26, num_windows, 200, num_channels)

        # **计算特征**
        features_batches = []
        for batch_idx in range(segments_array.shape[0]):  # 26 个 batch
            batch_features = []  # 存储当前 batch 的所有窗口特征
            for window_idx in range(segments_array.shape[1]):  # 计算每个窗口
                window = segments_array[batch_idx, window_idx]  # (200, num_channels)
                features = extract_features(window)  # 计算 (15, 10)
                batch_features.append(features)  # 存入 batch

            features_batches.append(np.array(batch_features))

        features_array = np.array(features_batches)  # (26, num_windows, 15, 10)

        # **存储数据**
        all_features.append(features_array)
        all_labels.extend(labels)  # 每个文件 26个 cycle，取 shuffle_order.xlsx 里的前 26 个标签

    # **最终转换为 NumPy 数组**
    all_features = np.vstack(all_features)  # 合并所有 batch，形状 (总 batch, num_windows, 15, 10)
    all_labels = np.array(all_labels)  # (总 batch,)

    # **保存**
    np.save(os.path.join(output_folder, "feature_matrix.npy"), all_features)
    np.save(os.path.join(output_folder, "labels.npy"), all_labels)

    print(f"Feature extraction complete! Shape: {all_features.shape}")
    print(f"Labels saved: {all_labels.shape}")
    print(f"Feature matrix saved at: {output_folder}/feature_matrix.npy")
    print(f"Labels saved at: {output_folder}/labels.npy")


def extract_features(segment):
    """
    计算 EMG 和 IMU 的 15 个特征，并拼接成 (15, 10)
    """
    # **分离 EMG (前四列) 和 IMU (后六列)**
    emg_signals = segment[:, :4]  # 4 个 EMG 通道 (200, 4)
    imu_signals = segment[:, 4:]  # 6 个 IMU 通道 (200, 6)

    def compute_emg_features(signal):
        return np.array([
            np.var(signal), np.mean(np.abs(signal)), np.sqrt(np.mean(signal**2)),
            np.std(signal), np.mean(np.abs(np.diff(signal))), np.max(signal),
            np.min(signal), np.sum(np.diff(signal) > 0), np.sum(np.diff(np.sign(signal)) != 0),
            stats.kurtosis(signal), stats.skew(signal), np.sum(np.abs(np.diff(signal))),
            np.sum(signal ** 2), np.log10(np.mean(signal**2) + 1e-10),
            np.mean(welch(signal, fs=1000, nperseg=200)[1])
        ])

    def compute_imu_features(signal):
        return np.array([
            np.var(signal), np.mean(signal), np.sqrt(np.mean(signal**2)),
            np.std(signal), np.max(signal), np.min(signal),
            stats.kurtosis(signal), stats.skew(signal),
            np.mean(np.abs(np.diff(signal))), np.sum(np.abs(np.diff(signal))),
            np.sum(signal ** 2), np.log10(np.mean(signal**2) + 1e-10),
            np.mean(welch(signal, fs=1000, nperseg=200)[1]), np.median(signal), np.ptp(signal)
        ])

    emg_features = np.array([compute_emg_features(emg_signals[:, i]) for i in range(4)]).T  # (15, 4)
    imu_features = np.array([compute_imu_features(imu_signals[:, i]) for i in range(6)]).T  # (15, 6)

    return np.concatenate((emg_features, imu_features), axis=1)  # (15, 10)

# **运行函数**
root = r"data\G"
input_folder = root
output_folder = os.path.join(root, "windowed_data")
shuffle_order_file = os.path.join(root,"shuffle_order.xlsx")
process_emg_folder(input_folder, output_folder, shuffle_order_file)


(15, 26)
Processing sensor_data1.csv (1/15)
Detected 10 channels (excluding time column).


C:\Users\user\AppData\Local\Temp\ipykernel_10696\1407476850.py:61: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       -3.775955
1       -3.775955
2       -0.214248
3       -0.214248
4       -0.214248
           ...   
45715   -0.005459
45716   -0.017740
45717   -0.017740
45718   -0.017740
45719   -0.017740
Name: EMG1, Length: 45720, dtype: float64' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  raw_data.iloc[:, 1:] = normalized_data.astype(np.float64)
C:\Users\user\AppData\Local\Temp\ipykernel_10696\1407476850.py:61: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       -6.768217
1       -6.768217
2        0.054180
3        0.076403
4        0.054180
           ...   
45715   -0.012488
45716   -0.012488
45717   -0.034711
45718   -0.056934
45719   -0.034711
Name: EMG2, Length: 45720, dtype: float64' h

Processing sensor_data10.csv (2/15)
Detected 10 channels (excluding time column).


C:\Users\user\AppData\Local\Temp\ipykernel_10696\1407476850.py:61: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       -5.279856
1       -5.279856
2       -0.008469
3       -0.008469
4       -0.008469
           ...   
46195   -0.043035
46196   -0.043035
46197   -0.043035
46198   -0.112168
46199   -0.146735
Name: EMG1, Length: 46200, dtype: float64' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  raw_data.iloc[:, 1:] = normalized_data.astype(np.float64)
C:\Users\user\AppData\Local\Temp\ipykernel_10696\1407476850.py:61: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       -7.967664
1       -7.967664
2        0.052054
3        0.025846
4        0.025846
           ...   
46195   -0.052779
46196   -0.052779
46197   -0.026571
46198   -0.000362
46199   -0.000362
Name: EMG2, Length: 46200, dtype: float64' h

Processing sensor_data11.csv (3/15)
Detected 10 channels (excluding time column).


...

Processing sensor_data8.csv (14/15)
Detected 10 channels (excluding time column).


C:\Users\user\AppData\Local\Temp\ipykernel_10696\1407476850.py:61: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       -4.303589
1       -4.303589
2       -0.032001
3       -0.032001
4       -0.032001
           ...   
45996   -0.088206
45997   -0.088206
45998   -0.088206
45999   -0.102257
46000   -0.088206
Name: EMG1, Length: 46001, dtype: float64' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  raw_data.iloc[:, 1:] = normalized_data.astype(np.float64)
C:\Users\user\AppData\Local\Temp\ipykernel_10696\1407476850.py:61: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       -6.680219
1       -6.680219
2       -0.269486
3       -0.269486
4       -0.269486
           ...   
45996    0.037878
45997    0.015923
45998    0.015923
45999    0.015923
46000   -0.006031
Name: EMG2, Length: 46001, dtype: float64' h

Processing sensor_data9.csv (15/15)
Detected 10 channels (excluding time column).


C:\Users\user\AppData\Local\Temp\ipykernel_10696\1407476850.py:61: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       -4.303005
1       -4.303005
2        0.506601
3        0.562690
4        0.604756
           ...   
46333   -0.096353
46334   -0.110375
46335   -0.096353
46336   -0.124397
46337   -0.096353
Name: EMG1, Length: 46338, dtype: float64' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  raw_data.iloc[:, 1:] = normalized_data.astype(np.float64)
C:\Users\user\AppData\Local\Temp\ipykernel_10696\1407476850.py:61: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       -7.636328
1       -7.636328
2       -0.081067
3       -0.030866
4       -0.005765
           ...   
46333   -0.005765
46334    0.019335
46335    0.019335
46336    0.044436
46337    0.044436
Name: EMG2, Length: 46338, dtype: float64' h

Feature extraction complete! Shape: (390, 49, 15, 10)
Labels saved: (390,)
Feature matrix saved at: data\G\windowed_data/feature_matrix.npy
Labels saved at: data\G\windowed_data/labels.npy


In [15]:
import numpy as np
import pandas as pd
import scipy.stats as stats
from scipy.signal import welch
import os

def process_emg_folder(input_folder, output_folder, shuffle_order_file):
    """
    遍历 input_folder 下的所有 CSV 文件，处理 EMG 和 IMU 数据，并保存特征矩阵和标签到 output_folder。
    """
    # 设定采样率
    original_fs = 1000  # 原采样率 Hz
    target_fs = 200  # 目标采样率 Hz
    downsample_factor = original_fs // target_fs  # 降采样因子
    cycle_duration = 10  # 每个周期 10 秒
    skip_seconds = 5  # 跳过前 4 秒
    use_seconds = 5  # 需要保留的秒数

    cycle_samples = (original_fs * cycle_duration) // downsample_factor  # 2000
    skip_samples = (original_fs * skip_seconds) // downsample_factor  # 1000
    use_samples = (original_fs * use_seconds) // downsample_factor  # 1000

    # 滑动窗口参数
    window_size = 100  # 200ms = 40 采样点 (原 200, 降采样后 40)
    step_size = 50  # 100ms = 20 采样点 (原 100, 降采样后 20)
    num_windows = (use_samples - window_size) // step_size + 1  # 计算窗口数

    # 创建存储文件夹
    os.makedirs(output_folder, exist_ok=True)

    # 读取 shuffle_order.xlsx
    shuffle_df = pd.read_excel(shuffle_order_file, engine="openpyxl")
    if shuffle_df.shape[0] < 15:
        raise ValueError("标签文件数据不足 15 组！请检查 `shuffle_order.xlsx`。")

    all_features = []
    all_labels = []
    
    csv_files = sorted([f for f in os.listdir(input_folder) if f.endswith(".csv")])

    for file_idx, file_name in enumerate(csv_files):
        file_path = os.path.join(input_folder, file_name)
        print(f"Processing {file_name} ({file_idx+1}/{len(csv_files)})")

        raw_data = pd.read_csv(file_path)
        num_channels = raw_data.shape[1] - 1
        print(f"Detected {num_channels} channels (excluding time column).")

        # 降采样
        raw_data = raw_data.iloc[::downsample_factor, :].reset_index(drop=True)
        
        # 归一化
        data = raw_data.iloc[:, 1:]
        mean_vals = data.mean(axis=0)
        std_vals = data.std(axis=0)
        normalized_data = (data - mean_vals) / (std_vals + 1e-10)
        raw_data.iloc[:, 1:] = normalized_data.astype(np.float64)

        segments = []
        labels = shuffle_df.iloc[file_idx].values.tolist()
        num_cycles = 26

        for i in range(num_cycles):
            start_idx = i * cycle_samples + skip_samples
            end_idx = start_idx + use_samples

            if end_idx > len(raw_data):
                segment = raw_data.iloc[start_idx:].values
                pad_size = use_samples - len(segment)
                segment = np.pad(segment, ((0, pad_size), (0, 0)), mode='constant', constant_values=0)
            else:
                segment = raw_data.iloc[start_idx:end_idx].values

            windows = [
                segment[j:j + window_size, 1:]
                for j in range(0, use_samples - window_size + 1, step_size)
            ]
            segments.append(np.array(windows))

        segments_array = np.array(segments)
        
        features_batches = []
        for batch_idx in range(segments_array.shape[0]):
            batch_features = []
            for window_idx in range(segments_array.shape[1]):
                window = segments_array[batch_idx, window_idx]
                features = extract_features(window)
                batch_features.append(features)

            features_batches.append(np.array(batch_features))

        features_array = np.array(features_batches)

        all_features.append(features_array)
        all_labels.extend(labels)

    all_features = np.vstack(all_features)
    all_labels = np.array(all_labels)

    np.save(os.path.join(output_folder, "feature_matrix.npy"), all_features)
    np.save(os.path.join(output_folder, "labels.npy"), all_labels)

    print(f"Feature extraction complete! Shape: {all_features.shape}")
    print(f"Labels saved: {all_labels.shape}")
    print(f"Feature matrix saved at: {output_folder}/feature_matrix.npy")
    print(f"Labels saved at: {output_folder}/labels.npy")

def extract_features(segment):
    emg_signals = segment[:, :4]
    imu_signals = segment[:, 4:]

    def compute_emg_features(signal):
        return np.array([
            np.var(signal), np.mean(np.abs(signal)), np.sqrt(np.mean(signal**2)),
            np.std(signal), np.mean(np.abs(np.diff(signal))), np.max(signal),
            np.min(signal), np.sum(np.diff(signal) > 0), np.sum(np.diff(np.sign(signal)) != 0),
            stats.kurtosis(signal), stats.skew(signal), np.sum(np.abs(np.diff(signal))),
            np.sum(signal ** 2), np.log10(np.mean(signal**2) + 1e-10),
            np.mean(welch(signal, fs=200, nperseg=40)[1])
        ])

    def compute_imu_features(signal):
        return np.array([
            np.var(signal), np.mean(signal), np.sqrt(np.mean(signal**2)),
            np.std(signal), np.max(signal), np.min(signal),
            stats.kurtosis(signal), stats.skew(signal),
            np.mean(np.abs(np.diff(signal))), np.sum(np.abs(np.diff(signal))),
            np.sum(signal ** 2), np.log10(np.mean(signal**2) + 1e-10),
            np.mean(welch(signal, fs=200, nperseg=40)[1]), np.median(signal), np.ptp(signal)
        ])

    emg_features = np.array([compute_emg_features(emg_signals[:, i]) for i in range(4)]).T
    imu_features = np.array([compute_imu_features(imu_signals[:, i]) for i in range(6)]).T

    return np.concatenate((emg_features, imu_features), axis=1)

root = r"data\G"
input_folder = root
output_folder = os.path.join(root, "windowed_data")
shuffle_order_file = os.path.join(root, "shuffle_order.xlsx")
process_emg_folder(input_folder, output_folder, shuffle_order_file)

ValueError: 标签文件数据不足 15 组！请检查 `shuffle_order.xlsx`。

In [5]:
import numpy as np
import os
from sklearn.model_selection import train_test_split

def load_data(data_folder, test_size=0.2, random_state=42):
    """
    加载 `feature_matrix.npy` 和 `labels.npy` 数据，并划分训练集和测试集。

    参数：
    - data_folder: 存放数据的文件夹路径
    - test_size: 测试集比例 (默认 20%)
    - random_state: 随机种子，保证可复现性

    返回：
    - X_train: 训练集特征，形状 (train_batches, num_windows, 15, num_channels)
    - X_test: 测试集特征，形状 (test_batches, num_windows, 15, num_channels)
    - y_train: 训练集标签，形状 (train_batches,)
    - y_test: 测试集标签，形状 (test_batches,)
    """
    # **加载数据**
    feature_path = os.path.join(data_folder, "feature_matrix.npy")
    label_path = os.path.join(data_folder, "labels.npy")

    if not os.path.exists(feature_path) or not os.path.exists(label_path):
        raise FileNotFoundError("特征文件或标签文件未找到，请检查路径！")

    X = np.load(feature_path)  # 形状 (num_batches, num_windows, 15, num_channels)
    y = np.load(label_path)  # 形状 (num_batches,)

    # **数据基本信息**
    print(f"Loaded features from {feature_path}, shape: {X.shape}")
    print(f"Loaded labels from {label_path}, shape: {y.shape}")

    # **划分训练集和测试集**
    # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state, stratify=y)
    X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=random_state
        )

    # **打印数据划分信息**
    print(f"Training set: X_train: {X_train.shape}, y_train: {y_train.shape}")
    print(f"Testing set: X_test: {X_test.shape}, y_test: {y_test.shape}")

    return X_train, X_test, y_train, y_test

# **使用示例**
data_folder = r"E:\MSC\Spring\AML\GestureLink\data\G\windowed_data"
X_train, X_test, y_train, y_test = load_data(data_folder)



Loaded features from E:\MSC\Spring\AML\GestureLink\data\G\windowed_data\feature_matrix.npy, shape: (390, 19, 15, 10)
Loaded labels from E:\MSC\Spring\AML\GestureLink\data\G\windowed_data\labels.npy, shape: (390,)
Training set: X_train: (312, 19, 15, 10), y_train: (312,)
Testing set: X_test: (78, 19, 15, 10), y_test: (78,)


----------------------------

# 处理分开的数据

In [23]:
import numpy as np
import pandas as pd
import os
import scipy.stats as stats
from scipy.signal import welch

def process_emg_folder(input_folder, output_folder, shuffle_order_file):
    """
    处理 EMG 数据文件，先按照 shuffle_order.xlsx 划分手势数据，
    然后进行窗口切分，并提取特征。
    """
    fs = 1000  # 采样率 1000Hz，每秒 1000 个数据点
    target_samples = 5000  # 每个手势的固定采样点数
    window_size = 200  # 窗口大小 400 采样点（0.4s）
    step_size = 100  # 窗口滑动步长 200 采样点（0.2s）

    # 创建输出文件夹
    os.makedirs(output_folder, exist_ok=True)

    # 读取 shuffle_order.xlsx
    shuffle_df = pd.read_excel(shuffle_order_file, engine="openpyxl")
    if shuffle_df.shape[1] < 26:
        raise ValueError("标签文件数据不足 26 组！请检查 `shuffle_order.xlsx`。")

    all_features = []
    all_labels = []
    csv_files = sorted([f for f in os.listdir(input_folder) if f.endswith(".csv")])

    for file_idx, file_name in enumerate(csv_files):
        file_path = os.path.join(input_folder, file_name)
        print(f"Processing {file_name} ({file_idx + 1}/{len(csv_files)})")

        # 读取数据
        raw_data = pd.read_csv(file_path)
        num_channels = raw_data.shape[1] - 1  # 去掉时间列
        print(f"Detected {num_channels} channels (excluding time column).")

        # 归一化数据
        data = raw_data.iloc[:, 1:]
        normalized_data = (data - data.mean(axis=0)) / (data.std(axis=0) + 1e-10)
        raw_data.iloc[:, 1:] = normalized_data.astype(np.float64)

        # 按 shuffle_order.xlsx 划分 26 组手势
        valid_segments = []
        for i in range(26):
            start_idx = shuffle_df.iloc[file_idx, i]  # 该手势起始索引
            if start_idx + target_samples <= len(raw_data):
                segment = raw_data.iloc[start_idx:start_idx + target_samples, 1:].values
            else:
                segment = raw_data.iloc[start_idx:, 1:].values
                pad_size = target_samples - len(segment)
                segment = np.pad(segment, ((0, pad_size), (0, 0)), mode='constant', constant_values=0)
            valid_segments.append(segment)

        valid_segments = np.array(valid_segments)  # 形状 (26, 5000, num_channels)

        # 进行窗口划分和特征提取
        features_batches = []
        for seg in valid_segments:
            windows = [seg[j:j + window_size, :] for j in range(0, target_samples - window_size + 1, step_size)]
            batch_features = np.array([extract_features(window) for window in windows])  # (num_windows, 15, num_channels)
            features_batches.append(batch_features)

        processed_segments = np.array(features_batches)  # (26, num_windows, 15, num_channels)

        if processed_segments.shape[0] != 26:
            print(f"Warning: {file_name} 提取到 {processed_segments.shape[0]} 个手势，不是 26 个。")

        # 存储数据
        all_features.append(processed_segments)
        all_labels.extend(shuffle_df.iloc[file_idx, :26].values.tolist())

    # 最终转换为 NumPy 数组
    all_features = np.vstack(all_features)  # (总手势数, num_windows, 15, num_channels)
    all_labels = np.array(all_labels)  # (总手势数,)

    # 保存数据
    np.save(os.path.join(output_folder, "feature_matrix.npy"), all_features)
    np.save(os.path.join(output_folder, "labels.npy"), all_labels)

    print(f"Feature extraction complete! Shape: {all_features.shape}")
    print(f"Labels saved: {all_labels.shape}")
    print(f"Feature matrix saved at: {output_folder}/feature_matrix.npy")
    print(f"Labels saved at: {output_folder}/labels.npy")

def extract_features(segment):
    """
    计算 EMG 和 IMU 的特征。
    """
    emg_signals = segment[:, :4]  # 4 个 EMG 通道
    imu_signals = segment[:, 4:]  # 6 个 IMU 通道

    def compute_emg_features(signal):
        return np.array([
            np.var(signal), np.mean(np.abs(signal)), np.sqrt(np.mean(signal**2)),
            np.std(signal), np.mean(np.abs(np.diff(signal))), np.max(signal),
            np.min(signal), np.sum(np.diff(signal) > 0), np.sum(np.diff(np.sign(signal)) != 0),
            stats.kurtosis(signal), stats.skew(signal), np.sum(np.abs(np.diff(signal))),
            np.sum(signal ** 2), np.log10(np.mean(signal**2) + 1e-10),
            np.mean(welch(signal, fs=1000, nperseg=200)[1])
        ])

    def compute_imu_features(signal):
        return np.array([
            np.var(signal), np.mean(signal), np.sqrt(np.mean(signal**2)),
            np.std(signal), np.max(signal), np.min(signal),
            stats.kurtosis(signal), stats.skew(signal),
            np.mean(np.abs(np.diff(signal))), np.sum(np.abs(np.diff(signal))),
            np.sum(signal ** 2), np.log10(np.mean(signal**2) + 1e-10),
            np.mean(welch(signal, fs=1000, nperseg=200)[1]), np.median(signal), np.ptp(signal)
        ])

    emg_features = np.array([compute_emg_features(emg_signals[:, i]) for i in range(4)]).T  # (15, 4)
    imu_features = np.array([compute_imu_features(imu_signals[:, i]) for i in range(6)]).T  # (15, 6)

    return np.concatenate((emg_features, imu_features), axis=1)  # (15, 10)

# 运行代码
root = r"data\FZH"
input_folder = root
output_folder = os.path.join(root, "processed_data")
shuffle_order_file = os.path.join(root, "shuffle_order.xlsx")
process_emg_folder(input_folder, output_folder, shuffle_order_file)

Processing sensor_data1.csv (1/15)
Detected 10 channels (excluding time column).


C:\Users\user\AppData\Local\Temp\ipykernel_10696\2422918298.py:41: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       -0.033795
1       -0.033795
2       -0.033795
3       -0.033795
4       -0.033795
           ...   
22275   -0.033795
22276   -0.033795
22277   -0.033795
22278   -0.033795
22279   -0.033795
Name: 303, Length: 22280, dtype: float64' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  raw_data.iloc[:, 1:] = normalized_data.astype(np.float64)
C:\Users\user\AppData\Local\Temp\ipykernel_10696\2422918298.py:41: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       -0.132256
1       -0.132256
2       -0.114027
3       -0.123142
4       -0.132256
           ...   
22275    1.079998
22276    1.079998
22277    1.061769
22278    1.061769
22279    1.070883
Name: 293, Length: 22280, dtype: float64' has

Processing sensor_data10.csv (2/15)
Detected 10 channels (excluding time column).


C:\Users\user\AppData\Local\Temp\ipykernel_10696\2422918298.py:41: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       -1.149433
1       -0.333118
2       -1.149433
3       -1.965747
4       -0.333118
           ...   
22307    2.115824
22308    2.115824
22309    2.115824
22310    2.115824
22311    2.115824
Name: 305, Length: 22312, dtype: float64' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  raw_data.iloc[:, 1:] = normalized_data.astype(np.float64)
C:\Users\user\AppData\Local\Temp\ipykernel_10696\2422918298.py:41: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       -0.707460
1       -0.964646
2       -1.178968
3       -1.250408
4       -1.321849
           ...   
22307    1.921554
22308    2.078723
22309    2.193028
22310    2.293045
22311    2.407350
Name: 272, Length: 22312, dtype: float64' has

Processing sensor_data11.csv (3/15)
Detected 10 channels (excluding time column).


...

Processing sensor_data8.csv (14/15)
Detected 10 channels (excluding time column).


C:\Users\user\AppData\Local\Temp\ipykernel_10696\2422918298.py:41: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       -0.105940
1       -0.105940
2       -0.105940
3       -0.105940
4       -0.105940
           ...   
22182    0.969801
22183   -0.105940
22184   -0.105940
22185   -0.105940
22186   -0.105940
Name: 306, Length: 22187, dtype: float64' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  raw_data.iloc[:, 1:] = normalized_data.astype(np.float64)
C:\Users\user\AppData\Local\Temp\ipykernel_10696\2422918298.py:41: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0        0.085506
1        0.085506
2        0.071930
3        0.071930
4        0.099082
           ...   
22182    0.126235
22183    0.112659
22184    0.112659
22185    0.139811
22186    0.139811
Name: 309, Length: 22187, dtype: float64' has

Processing sensor_data9.csv (15/15)
Detected 10 channels (excluding time column).


C:\Users\user\AppData\Local\Temp\ipykernel_10696\2422918298.py:41: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       -0.327510
1       -0.327510
2       -0.327510
3        0.968289
4       -0.327510
           ...   
22377   -0.327510
22378   -0.327510
22379   -0.327510
22380    0.968289
22381    0.968289
Name: 306, Length: 22382, dtype: float64' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  raw_data.iloc[:, 1:] = normalized_data.astype(np.float64)
C:\Users\user\AppData\Local\Temp\ipykernel_10696\2422918298.py:41: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       -0.079637
1       -0.091994
2       -0.091994
3       -0.091994
4       -0.079637
           ...   
22377   -0.549196
22378   -0.598623
22379   -0.623337
22380   -0.635694
22381   -0.685121
Name: 300, Length: 22382, dtype: float64' has

Feature extraction complete! Shape: (390, 49, 15, 10)
Labels saved: (390,)
Feature matrix saved at: data\FZH\processed_data/feature_matrix.npy
Labels saved at: data\FZH\processed_data/labels.npy


In [12]:
import numpy as np
import os
from sklearn.model_selection import train_test_split

def load_data(data_folder, test_size=0.2, random_state=42):
    """
    加载 `feature_matrix.npy` 和 `labels.npy` 数据，并划分训练集和测试集。

    参数：
    - data_folder: 存放数据的文件夹路径
    - test_size: 测试集比例 (默认 20%)
    - random_state: 随机种子，保证可复现性

    返回：
    - X_train: 训练集特征，形状 (train_batches, num_windows, 15, num_channels)
    - X_test: 测试集特征，形状 (test_batches, num_windows, 15, num_channels)
    - y_train: 训练集标签，形状 (train_batches,)
    - y_test: 测试集标签，形状 (test_batches,)
    """
    # **加载数据**
    feature_path = os.path.join(data_folder, "feature_matrix.npy")
    label_path = os.path.join(data_folder, "labels.npy")

    if not os.path.exists(feature_path) or not os.path.exists(label_path):
        raise FileNotFoundError("特征文件或标签文件未找到，请检查路径！")

    X = np.load(feature_path)  # 形状 (num_batches, num_windows, 15, num_channels)
    y = np.load(label_path)  # 形状 (num_batches,)

    # **数据基本信息**
    print(f"Loaded features from {feature_path}, shape: {X.shape}")
    print(f"Loaded labels from {label_path}, shape: {y.shape}")

    # **划分训练集和测试集**
    # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state, stratify=y)
    X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=random_state
        )

    # **打印数据划分信息**
    print(f"Training set: X_train: {X_train.shape}, y_train: {y_train.shape}")
    print(f"Testing set: X_test: {X_test.shape}, y_test: {y_test.shape}")

    return X_train, X_test, y_train, y_test

# **使用示例**
data_folder = r"E:\MSC\Spring\AML\GestureLink\data\FZH\processed_data"
X_train, X_test, y_train, y_test = load_data(data_folder)



Loaded features from E:\MSC\Spring\AML\GestureLink\data\FZH\processed_data\feature_matrix.npy, shape: (390, 49, 15, 10)
Loaded labels from E:\MSC\Spring\AML\GestureLink\data\FZH\processed_data\labels.npy, shape: (390,)
Training set: X_train: (312, 49, 15, 10), y_train: (312,)
Testing set: X_test: (78, 49, 15, 10), y_test: (78,)


KNN

In [14]:
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report

def load_data(data_folder, test_size=0.2, random_state=42):
    """ 加载 MyoWare EMG 特征数据，并拆分训练集和测试集 """
    feature_path = os.path.join(data_folder, "feature_matrix.npy")
    label_path = os.path.join(data_folder, "labels.npy")

    if not os.path.exists(feature_path) or not os.path.exists(label_path):
        raise FileNotFoundError("特征文件或标签文件未找到！")

    X = np.load(feature_path)  # 形状: (num_batches, num_windows, 15, num_channels)

    # X = X[:,:,:,:6]
    y = np.load(label_path)    # 形状: (num_batches,)

    print(f"加载特征矩阵: {X.shape}")
    print(f"加载标签: {y.shape}")

    # **展平数据**: (num_batches, num_windows, 15, num_channels) → (num_batches, num_windows * 15 * num_channels)
    num_batches, num_windows, num_rows, num_channels = X.shape
    X = X.reshape(num_batches, -1)  # 变成 (num_batches, num_features)

    # **确保每个类别都有足够样本**
    num_classes = len(np.unique(y))
    test_size = max(num_classes, int(len(y) * test_size))  # 至少保证每个类别有样本

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state, stratify=y
    )

    print(f"训练集: {X_train.shape}, 测试集: {X_test.shape}")
    return X_train, X_test, y_train, y_test

# **加载数据**
# **使用示例**
data_folder = r"E:\MSC\Spring\AML\GestureLink\data\FZH\processed_data"
X_train, X_test, y_train, y_test = load_data(data_folder)

# **检查并去除 NaN**
print(f"NaNs in X_train: {np.isnan(X_train).sum()} / {X_train.size}")
print(f"NaNs in X_test: {np.isnan(X_test).sum()} / {X_test.size}")

# 过滤 NaN 样本
mask_train = ~np.isnan(X_train).any(axis=1)
mask_test = ~np.isnan(X_test).any(axis=1)

X_train, y_train = X_train[mask_train], y_train[mask_train]
X_test, y_test = X_test[mask_test], y_test[mask_test]
# print(X_train.shape, y_train.shape)
# **再次检查 NaN**
print(f"NaNs in X_train (after cleaning): {np.isnan(X_train).sum()} / {X_train.size}")
print(f"NaNs in X_test (after cleaning): {np.isnan(X_test).sum()} / {X_test.size}")

# **KNN 训练**
knn = KNeighborsClassifier(n_neighbors=5)  # k=5
knn.fit(X_train, y_train)

# **KNN 预测**
y_pred = knn.predict(X_test)

# **评估**
accuracy = accuracy_score(y_test, y_pred)
print(f"KNN 测试集准确率: {accuracy:.2f}")

# **打印分类报告**
print("分类报告:\n", classification_report(y_test, y_pred))


加载特征矩阵: (390, 49, 15, 10)
加载标签: (390,)
训练集: (312, 7350), 测试集: (78, 7350)
NaNs in X_train: 0 / 2293200
NaNs in X_test: 0 / 573300
NaNs in X_train (after cleaning): 0 / 2293200
NaNs in X_test (after cleaning): 0 / 573300
KNN 测试集准确率: 0.00
分类报告:
               precision    recall  f1-score   support

           1       0.00      0.00      0.00       3.0
           2       0.00      0.00      0.00       3.0
           3       0.00      0.00      0.00       3.0
           4       0.00      0.00      0.00       3.0
           5       0.00      0.00      0.00       3.0
           6       0.00      0.00      0.00       3.0
           7       0.00      0.00      0.00       3.0
           8       0.00      0.00      0.00       3.0
           9       0.00      0.00      0.00       3.0
          10       0.00      0.00      0.00       3.0
          11       0.00      0.00      0.00       3.0
          12       0.00      0.00      0.00       3.0
          13       0.00      0.00      0.00       3.0


d:\MY_SOFTWARE\Anaconda\envs\tensorflow\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\MY_SOFTWARE\Anaconda\envs\tensorflow\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\MY_SOFTWARE\Anaconda\envs\tensorflow\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(resu

Libsvm

In [15]:
import numpy as np
import os
from sklearn.model_selection import train_test_split
from libsvm.svmutil import *

def load_data(data_folder, test_size=0.2, random_state=42):
    """
    加载 `feature_matrix.npy` 和 `labels.npy` 数据，并划分训练集和测试集。

    参数：
    - data_folder: 存放数据的文件夹路径
    - test_size: 测试集比例 (默认 20%)
    - random_state: 随机种子，保证可复现性

    返回：
    - X_train: 训练集特征，形状 (train_samples, num_features)
    - X_test: 测试集特征，形状 (test_samples, num_features)
    - y_train: 训练集标签，形状 (train_samples,)
    - y_test: 测试集标签，形状 (test_samples,)
    """
    feature_path = os.path.join(data_folder, "feature_matrix.npy")
    label_path = os.path.join(data_folder, "labels.npy")

    if not os.path.exists(feature_path) or not os.path.exists(label_path):
        raise FileNotFoundError("特征文件或标签文件未找到，请检查路径！")

    X = np.load(feature_path)  # 形状 (num_batches, num_windows, 15, num_channels)
    y = np.load(label_path)  # 形状 (num_batches,)

    print(f"Loaded features from {feature_path}, shape: {X.shape}")
    print(f"Loaded labels from {label_path}, shape: {y.shape}")

    # **展平数据 (num_batches, num_windows, 15, num_channels) -> (num_batches, num_windows * 15 * num_channels)**
    num_batches, num_windows, height, num_channels = X.shape
    X = X.reshape(num_batches, -1)  # 变成 (num_batches, num_windows * 15 * num_channels)

    # **划分训练集和测试集**
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    print(f"Training set: X_train: {X_train.shape}, y_train: {y_train.shape}")
    print(f"Testing set: X_test: {X_test.shape}, y_test: {y_test.shape}")

    return X_train, X_test, y_train, y_test

# **数据路径**
data_folder = r"E:\MSC\Spring\AML\GestureLink\data\FZH\processed_data"
X_train, X_test, y_train, y_test = load_data(data_folder)

# **转换数据格式，适应 LIBSVM**
X_train_list = [dict(enumerate(x, 1)) for x in X_train]  # 将 NumPy 数组转换为 LIBSVM 格式的字典列表
X_test_list = [dict(enumerate(x, 1)) for x in X_test]  # 同样转换测试集

# **训练 SVM 模型**
svm_model = svm_train(y_train.tolist(), X_train_list, '-s 0 -t 2 -c 1')  # -s 0: C-SVC, -t 2: RBF 核, -c 1: 惩罚参数C

# **在测试集上评估**
pred_labels, acc, vals = svm_predict(y_test.tolist(), X_test_list, svm_model)

print(f"测试集准确率: {acc[0]:.2f}%")


Loaded features from E:\MSC\Spring\AML\GestureLink\data\FZH\processed_data\feature_matrix.npy, shape: (390, 49, 15, 10)
Loaded labels from E:\MSC\Spring\AML\GestureLink\data\FZH\processed_data\labels.npy, shape: (390,)
Training set: X_train: (312, 7350), y_train: (312,)
Testing set: X_test: (78, 7350), y_test: (78,)
Accuracy = 0% (0/78) (classification)
测试集准确率: 0.00%


LSTM

In [8]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, TimeDistributed, Conv1D, Flatten
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling1D


# **加载数据**
data_folder = r"E:\MSC\Spring\AML\GestureLink\data\FZH\processed_data"
X_train, X_test, y_train, y_test = load_data(data_folder)
# X_train, X_test, y_train, y_test = load_and_split_data(data_folder, selected_labels=[24, 25, 26])
X_train = np.nan_to_num(X_train, nan=0.0)
X_test = np.nan_to_num(X_test, nan=0.0)
y_train = np.nan_to_num(y_train, nan = 0.0)
y_test = np.nan_to_num(y_test, nan = 0.0)
# **获取数据形状**
num_batches = X_train.shape[0]  # batch 维度
num_windows = X_train.shape[1]  # 时间步（窗口数 59）
num_features = X_train.shape[2]  # 特征数（15）
num_channels = X_train.shape[3]  # 通道数（1）

# **检测类别数量**
unique_classes = np.unique(y_train)
num_classes = len(unique_classes)  # 确保类别数正确

print(f"Corrected Classes: {num_classes}, Batches: {num_batches}, Windows: {num_windows}, Features: {num_features}, Channels: {num_channels}")

# **保持 X 形状**
X_train = X_train.reshape(num_batches, num_windows, num_features, num_channels)
X_test = X_test.reshape(X_test.shape[0], num_windows, num_features, num_channels)
X_train = X_train[:,:,:,:]
X_test = X_test[:,:,:,:]

# print(X_train.shape)

num_features = X_train.shape[2] 
num_channels = X_train.shape[3]
# **标签编码**
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)  # 转换成 0,1,2
y_test = encoder.transform(y_test)

# **转换为 One-Hot**
y_train = to_categorical(y_train, num_classes=num_classes)
y_test = to_categorical(y_test, num_classes=num_classes)

print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")  # (batch, 59, 15, 1) (batch, 3)
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")  # (batch, 59, 15, 1) (batch, 3)


# model = Sequential([
#     TimeDistributed(Flatten(), input_shape=(num_windows, 4, 10)),  # 把 (4, 10) 展平成 40
#     LSTM(64, return_sequences=True),
#     Dropout(0.3),
#     LSTM(32, return_sequences=False),
#     Dropout(0.3),
#     Dense(32, activation='relu'),
#     Dense(num_classes, activation='softmax')  # 多分类
# ])

model = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu', padding='same', input_shape=(num_windows, num_features * num_channels)),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),

    Conv1D(filters=128, kernel_size=3, activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),

    Conv1D(filters=256, kernel_size=3, activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),

    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(num_classes, activation='softmax')  # 多分类输出
])

X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], -1)  # (batch_size, 29, 40)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], -1)  

# # print(num_features)
# # **构建 LSTM 处理通道的模型**

# model = Sequential([
#     TimeDistributed(Conv1D(filters=16, kernel_size=3, activation='relu'), input_shape=(num_windows, num_features, num_channels)),  
#     TimeDistributed(GlobalAveragePooling1D()),  # 只对特征维度池化，不影响时间维度
#     LSTM(64, return_sequences=True),  # 保持时间序列结构
#     Dropout(0.2),
#     LSTM(128, return_sequences=False),  # 输出 2D (batch, 128)
#     Dropout(0.2),
#     Dense(64, activation='relu'),
#     Dropout(0.2),
#     Dense(num_classes, activation='softmax')  # 最终分类
# ])


# **编译模型**
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# **训练模型**
epochs = 40

batch_size = 8

history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=epochs,
    batch_size=batch_size,
    verbose=1
)

# **测试模型**
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

# **计算准确率**
accuracy = accuracy_score(y_test_classes, y_pred_classes)
print(f"Test Accuracy: {accuracy:.4f}")

# **保存模型**
# model.save("rnn_emg_model.h5")
# print("Model saved as rnn_emg_model.h5")


Loaded features from E:\MSC\Spring\AML\GestureLink\data\FZH\processed_data\feature_matrix.npy, shape: (390, 49, 15, 10)
Loaded labels from E:\MSC\Spring\AML\GestureLink\data\FZH\processed_data\labels.npy, shape: (390,)
Training set: X_train: (312, 49, 15, 10), y_train: (312,)
Testing set: X_test: (78, 49, 15, 10), y_test: (78,)
Corrected Classes: 26, Batches: 312, Windows: 49, Features: 15, Channels: 10
X_train shape: (312, 49, 15, 10), y_train shape: (312, 26)
X_test shape: (78, 49, 15, 10), y_test shape: (78, 26)
Epoch 1/40
39/39 [==============================] - 5s 27ms/step - loss: 4.7450 - accuracy: 0.0321 - val_loss: 3.3884 - val_accuracy: 0.0256
Epoch 2/40
39/39 [==============================] - 1s 15ms/step - loss: 4.0524 - accuracy: 0.0321 - val_loss: 3.2955 - val_accuracy: 0.0513
Epoch 3/40
39/39 [==============================] - 1s 14ms/step - loss: 3.7391 - accuracy: 0.0417 - val_loss: 3.3048 - val_accuracy: 0.0000e+00
Epoch 4/40
39/39 [==============================] - 

———————————————————————————————————

WXR

In [62]:
import numpy as np
import pandas as pd
import os
import scipy.stats as stats
from scipy.signal import welch

def process_emg_folder(input_folder, output_folder, shuffle_order_file):
    """
    处理 EMG 数据文件，自动选择 `shuffle_order.xlsx` 中的 5 个手势类别，
    然后进行窗口切分，并提取特征。
    """
    fs = 1000  # 采样率 1000Hz
    target_samples = 5000  # 每个手势的固定采样点数
    window_size = 200  # 窗口大小 200 采样点（0.2s）
    step_size = 100  # 窗口滑动步长 100 采样点（0.1s）

    # 创建输出文件夹
    os.makedirs(output_folder, exist_ok=True)

    # 读取 shuffle_order.xlsx
    shuffle_df = pd.read_excel(shuffle_order_file, engine="openpyxl")

    # **检查 shuffle_order.xlsx 是否有 NaN**
    if shuffle_df.isnull().values.any():
        print("Warning: `shuffle_order.xlsx` 解析后包含 NaN 值，已填充为 -1")
        shuffle_df = shuffle_df.fillna(-1)  # 避免 NaN 影响索引匹配

    # **自动提取 5 个唯一的类别**
    unique_labels = shuffle_df.iloc[:, :].values.flatten()
    unique_labels = np.unique(unique_labels)  # 获取所有唯一的类别
    unique_labels = unique_labels[unique_labels != -1]  # 移除 -1 (填充的无效数据)
    unique_labels = unique_labels.astype(int)  # 转换为整数
    
    if len(unique_labels) < 5:
        raise ValueError("数据中可用的手势类别不足 5 个，请检查 `shuffle_order.xlsx`。")
    
    selected_classes = unique_labels[:5]  # 选择前 5 个类别
    print(f"自动选择的 5 个手势类别: {selected_classes}")

    all_features = []
    all_labels = []
    csv_files = sorted([f for f in os.listdir(input_folder) if f.endswith(".csv")])

    for file_idx, file_name in enumerate(csv_files):
        if file_idx >= len(shuffle_df):
            print(f"Warning: `shuffle_order.xlsx` 只有 {len(shuffle_df)} 行，但尝试访问第 {file_idx} 行。跳过该文件。")
            continue  # 跳过越界文件

        file_path = os.path.join(input_folder, file_name)
        print(f"Processing {file_name} ({file_idx + 1}/{len(csv_files)})")

        # 读取数据
        raw_data = pd.read_csv(file_path)
        num_channels = raw_data.shape[1] - 1  # 去掉时间列
        print(f"Detected {num_channels} channels (excluding time column).")

        # **确保数据是数值型**
        raw_data.iloc[:, 1:] = raw_data.iloc[:, 1:].astype(np.float64)

        # 归一化数据
        data = raw_data.iloc[:, 1:]
        normalized_data = (data - data.mean(axis=0)) / (data.std(axis=0) + 1e-10)
        raw_data.iloc[:, 1:] = normalized_data.astype(np.float64)

        # 仅保留选定类别的数据
        valid_segments = []
        valid_labels = []
        for i in selected_classes:
            start_idx = np.where(shuffle_df.iloc[file_idx, :].values == i)[0]  # 找到匹配的索引
            if len(start_idx) == 0:
                print(f"Warning: 文件 {file_name} 未找到类别 {i}，跳过该类别。")
                continue  # 该 CSV 文件可能没有这个类别，跳过

            start_idx = shuffle_df.iloc[file_idx, start_idx[0]]  # 获取具体索引
            if start_idx + target_samples <= len(raw_data):
                segment = raw_data.iloc[start_idx:start_idx + target_samples, 1:].values
            else:
                segment = raw_data.iloc[start_idx:, 1:].values
                pad_size = target_samples - len(segment)
                segment = np.pad(segment, ((0, pad_size), (0, 0)), mode='constant', constant_values=0)

            # **确保 segment 仅包含数值**
            segment = segment.astype(np.float64)

            valid_segments.append(segment)
            valid_labels.append(i)  # 记录手势类别

        if len(valid_segments) == 0:
            print(f"Warning: {file_name} 没有匹配的手势数据，跳过该文件。")
            continue

        valid_segments = np.array(valid_segments)  # 形状 (5, 5000, num_channels)

        # 进行窗口划分和特征提取
        features_batches = []
        for seg in valid_segments:
            windows = [seg[j:j + window_size, :] for j in range(0, target_samples - window_size + 1, step_size)]
            batch_features = np.array([extract_features(window) for window in windows])  # (num_windows, 15, num_channels)
            features_batches.append(batch_features)

        processed_segments = np.array(features_batches)  # (5, num_windows, 15, num_channels)

        # 存储数据
        all_features.append(processed_segments)
        all_labels.extend(valid_labels)

    # 最终转换为 NumPy 数组
    if len(all_features) == 0:
        raise ValueError("未提取到任何手势数据，请检查 `shuffle_order.xlsx` 和 CSV 文件。")

    all_features = np.vstack(all_features)  # (总手势数, num_windows, 15, num_channels)
    all_labels = np.array(all_labels)  # (总手势数,)

    # 保存数据
    np.save(os.path.join(output_folder, "feature_matrix.npy"), all_features)
    np.save(os.path.join(output_folder, "labels.npy"), all_labels)

    print(f"Feature extraction complete! Shape: {all_features.shape}")
    print(f"Labels saved: {all_labels.shape}")
    print(f"Feature matrix saved at: {output_folder}/feature_matrix.npy")
    print(f"Labels saved at: {output_folder}/labels.npy")

# 运行代码
root = r"data\WXR_5"
input_folder = root
output_folder = os.path.join(root, "processed_data")
shuffle_order_file = os.path.join(root, "shuffle_order.xlsx")

# 自动选择 `shuffle_order.xlsx` 中的 5 个类别
process_emg_folder(input_folder, output_folder, shuffle_order_file)


自动选择的 5 个手势类别: [1 2 3 4 5]
Processing sensor_data1.csv (1/15)
Detected 10 channels (excluding time column).


C:\Users\user\AppData\Local\Temp\ipykernel_11656\3898272709.py:58: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       333.0
1       332.0
2       330.0
3       329.0
4       330.0
        ...  
4221    346.0
4222    343.0
4223    336.0
4224    328.0
4225    325.0
Name: 334, Length: 4226, dtype: float64' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  raw_data.iloc[:, 1:] = raw_data.iloc[:, 1:].astype(np.float64)
C:\Users\user\AppData\Local\Temp\ipykernel_11656\3898272709.py:58: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       304.0
1       304.0
2       303.0
3       304.0
4       305.0
        ...  
4221    268.0
4222    271.0
4223    271.0
4224    272.0
4225    275.0
Name: 302, Length: 4226, dtype: float64' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
 

Processing sensor_data10.csv (2/15)
Detected 10 channels (excluding time column).


C:\Users\user\AppData\Local\Temp\ipykernel_11656\3898272709.py:58: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       305.0
1       305.0
2       304.0
3       306.0
4       307.0
        ...  
4312    312.0
4313    311.0
4314    311.0
4315    311.0
4316    311.0
Name: 304, Length: 4317, dtype: float64' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  raw_data.iloc[:, 1:] = raw_data.iloc[:, 1:].astype(np.float64)
C:\Users\user\AppData\Local\Temp\ipykernel_11656\3898272709.py:58: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       305.0
1       304.0
2       295.0
3       300.0
4       306.0
        ...  
4312    283.0
4313    274.0
4314    269.0
4315    275.0
4316    275.0
Name: 296, Length: 4317, dtype: float64' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
 

Processing sensor_data11.csv (3/15)
Detected 10 channels (excluding time column).


...

Processing sensor_data6.csv (12/15)
Detected 10 channels (excluding time column).


C:\Users\user\AppData\Local\Temp\ipykernel_11656\3898272709.py:58: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       276.0
1       278.0
2       281.0
3       282.0
4       283.0
        ...  
4292    304.0
4293    303.0
4294    301.0
4295    302.0
4296    302.0
Name: 274, Length: 4297, dtype: float64' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  raw_data.iloc[:, 1:] = raw_data.iloc[:, 1:].astype(np.float64)
C:\Users\user\AppData\Local\Temp\ipykernel_11656\3898272709.py:58: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       209.0
1       206.0
2       214.0
3       218.0
4       214.0
        ...  
4292    312.0
4293    308.0
4294    299.0
4295    299.0
4296    299.0
Name: 209, Length: 4297, dtype: float64' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
 

Processing sensor_data7.csv (13/15)
Detected 10 channels (excluding time column).
Processing sensor_data8.csv (14/15)
Detected 10 channels (excluding time column).


C:\Users\user\AppData\Local\Temp\ipykernel_11656\3898272709.py:58: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       280.0
1       281.0
2       284.0
3       286.0
4       285.0
        ...  
4240    306.0
4241    307.0
4242    308.0
4243    307.0
4244    306.0
Name: 281, Length: 4245, dtype: float64' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  raw_data.iloc[:, 1:] = raw_data.iloc[:, 1:].astype(np.float64)
C:\Users\user\AppData\Local\Temp\ipykernel_11656\3898272709.py:58: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       250.0
1       241.0
2       245.0
3       250.0
4       242.0
        ...  
4240    311.0
4241    314.0
4242    320.0
4243    314.0
4244    313.0
Name: 247, Length: 4245, dtype: float64' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
 

Feature extraction complete! Shape: (70, 49, 15, 10)
Labels saved: (70,)
Feature matrix saved at: data\WXR_5\processed_data/feature_matrix.npy
Labels saved at: data\WXR_5\processed_data/labels.npy


_________________________________________________

In [74]:
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report

def load_data(data_folder, test_size=0.2, random_state=42):
    """
    加载 `feature_matrix.npy` 和 `labels.npy`，并拆分训练集和测试集。
    只保留自动选定的 5 个类别的数据。
    """
    feature_path = os.path.join(data_folder, "feature_matrix.npy")
    label_path = os.path.join(data_folder, "labels.npy")

    if not os.path.exists(feature_path) or not os.path.exists(label_path):
        raise FileNotFoundError("特征文件或标签文件未找到！")

    # 加载数据
    X = np.load(feature_path)  # 形状: (num_samples, num_windows, 15, num_channels)
    # X = X[:,:,:,:]
    y = np.load(label_path)    # 形状: (num_samples,)

    print(f"加载特征矩阵: {X.shape}")
    print(f"加载标签: {y.shape}")

    # **检查 NaN**
    total_nan = np.isnan(X).sum()
    print(f"数据集中 NaN 总数: {total_nan}")

    if total_nan == X.size:
        raise ValueError("所有数据都是 NaN，检查 `feature_matrix.npy` 是否正确。")

    # **用 0 替换 NaN**
    X = np.nan_to_num(X, nan=0.0)  # 把 NaN 变成 0，避免过滤掉所有数据

    # **展平数据**: (num_samples, num_windows, 15, num_channels) → (num_samples, num_features)
    num_samples, num_windows, num_rows, num_channels = X.shape
    X = X.reshape(num_samples, -1)  # 变成 (num_samples, num_features)

    # **确保每个类别都有足够样本**
    unique_classes, class_counts = np.unique(y, return_counts=True)
    print(f"数据集中唯一类别: {unique_classes}")
    print(f"每个类别的样本数量: {dict(zip(unique_classes, class_counts))}")

    if len(unique_classes) < 5:
        raise ValueError("数据类别不足 5 个，请检查 `shuffle_order.xlsx` 是否正确。")

    # **避免类别样本太少导致 stratify 失败**
    min_class_count = min(class_counts)
    if min_class_count < 2:
        print("Warning: 某些类别样本太少，禁用 `stratify`")
        stratify = None
    else:
        stratify = y

    # **拆分训练集和测试集**
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state, stratify=stratify
    )

    print(f"训练集: {X_train.shape}, 测试集: {X_test.shape}")

    # **检查数据是否仍然为空**
    if X_train.shape[0] == 0 or X_test.shape[0] == 0:
        raise ValueError("训练集或测试集为空，可能因为数据中存在过多 NaN，请检查 `feature_matrix.npy`。")

    return X_train, X_test, y_train, y_test

# **使用示例**
data_folder = r"data\WXR_5\processed_data"
X_train, X_test, y_train, y_test = load_data(data_folder)

# **KNN 训练**
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

# **KNN 预测**
y_pred = knn.predict(X_test)

# **评估**
accuracy = accuracy_score(y_test, y_pred)
print(f"KNN 测试集准确率: {accuracy:.2f}")

# **打印分类报告**
print("分类报告:\n", classification_report(y_test, y_pred))


In [76]:
# **转换数据格式，适应 LIBSVM**
X_train_list = [dict(enumerate(x, 1)) for x in X_train]  # 将 NumPy 数组转换为 LIBSVM 格式的字典列表
X_test_list = [dict(enumerate(x, 1)) for x in X_test]  # 同样转换测试集

# **训练 SVM 模型**
svm_model = svm_train(y_train.tolist(), X_train_list, '-s 0 -t 2 -c 1')  # -s 0: C-SVC, -t 2: RBF 核, -c 1: 惩罚参数C

# **在测试集上评估**
pred_labels, acc, vals = svm_predict(y_test.tolist(), X_test_list, svm_model)

print(f"测试集准确率: {acc[0]:.2f}%")


Accuracy = 7.14286% (1/14) (classification)
测试集准确率: 7.14%


In [ ]:
# **转换数据格式，适应 LIBSVM**
X_train_list = [dict(enumerate(x, 1)) for x in X_train]  # 将 NumPy 数组转换为 LIBSVM 格式的字典列表
X_test_list = [dict(enumerate(x, 1)) for x in X_test]  # 同样转换测试集

# **训练 SVM 模型**
svm_model = svm_train(y_train.tolist(), X_train_list, '-s 0 -t 2 -c 1')  # -s 0: C-SVC, -t 2: RBF 核, -c 1: 惩罚参数C

# **在测试集上评估**
pred_labels, acc, vals = svm_predict(y_test.tolist(), X_test_list, svm_model)

print(f"测试集准确率: {acc[0]:.2f}%")


Accuracy = 7.14286% (1/14) (classification)
测试集准确率: 7.14%
